In [ ]:
import os
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin


def cargar_datos():
    data_path = '../data'
    files = os.listdir(data_path)
    
    df_list = []

    for file in files:
        if file.endswith('.csv'):  
            file_path = os.path.join(data_path, file)

            parts = file.split('_')

            if len(parts) < 3:
                continue
            
            year = parts[0]  
            month_name = parts[2]  

            #if year not in ['2023', '2024']:
            if year not in ['2024']:

                continue

            #if month_name not in ['Gener', 'Febrer', 'Marc', 'Abril', 'Maig']:
            if month_name not in ['Maig']:

                continue

            try:
                #df = pd.read_csv(file_path, low_memory=True, dtype=str, skiprows=lambda i: i % 1000 != 0)
                df = pd.read_csv(file_path, low_memory=True, dtype=str)


                df['last_reported'] = pd.to_datetime(df['last_reported'], unit='s', errors='coerce')

                df['year'] = df['last_reported'].dt.year
                df['month'] = df['last_reported'].dt.month
                df['day'] = df['last_reported'].dt.day
                df['hour'] = df['last_reported'].dt.hour

                if 'traffic' in df.columns:
                    df.drop(columns=['traffic'], inplace=True)
                
                if 'V1' in df.columns:
                    df.drop(columns=['V1'], inplace=True)

                # Convertir columnas a float
                for col in ['num_bikes_available','num_docks_available','num_bikes_available_types.mechanical', 'num_bikes_available_types.ebike']:
                    df[col] = df[col].astype('float64')

                df_list.append(df)

            except Exception:
                continue

    merged_df = pd.concat(df_list, ignore_index=True) if df_list else None

    if merged_df is not None:
        df_2 = pd.read_csv('../Informacio_Estacions_Bicing_2025.csv', usecols=['station_id', 'lat', 'lon','capacity'], low_memory=False)

        merged_df['station_id'] = merged_df['station_id'].astype('Int64')
        df_2['station_id'] = df_2['station_id'].astype('Int64')
        
        merged_df = merged_df.merge(df_2, on='station_id', how='inner')

    # Crear columna sum_capacity
    merged_df['sum_capacity'] = merged_df['num_bikes_available'] + merged_df['num_docks_available']

    # Calcular la mediana de sum_capacity por estación
    median_capacity = merged_df.groupby('station_id')['sum_capacity'].median()

    def impute_capacity(row):
        return median_capacity[row['station_id']] if pd.isna(row['capacity']) else row['capacity']

    merged_df['capacity'] = merged_df.apply(impute_capacity, axis=1)
    # merged_df['diff_capacity_available'] = merged_df['capacity'] - (merged_df['num_bikes_available'] + merged_df['num_docks_available'])

    # Asegurar límites de num_docks_available
    merged_df['num_docks_available'] = merged_df.apply(
        lambda row: min(max(row['num_docks_available'], 0), row['capacity']),
        axis=1
    )

    # Crear columna target (% de bicis disponibles)
    merged_df['target'] = merged_df['num_docks_available'] / merged_df['capacity']

    # **Agrupar a nivel de hora por estación**
    aggregated_df = merged_df.groupby(['station_id', 'year', 'month', 'day', 'hour']).agg(
        num_bikes_available=('num_bikes_available', 'mean'),
        num_docks_available=('num_docks_available', 'mean'),
        num_mechanical=('num_bikes_available_types.mechanical', 'median'),
        num_ebike=('num_bikes_available_types.ebike', 'median'),
        target=('target', 'mean'),
        lat=('lat', 'first'),
        lon=('lon', 'first'),
        capacity=('capacity', 'first')
    ).reset_index()

    id = pd.read_csv('../data/metadata_sample_submission_2025.csv')

    llista_stations = pd.unique(id['station_id'])

    aggregated_df = aggregated_df[aggregated_df['station_id'].isin(llista_stations)]

    return aggregated_df

In [2]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

class CyclicalFeaturesTransformer(BaseEstimator, TransformerMixin):
    """
    Transformer para crear variables cíclicas (seno y coseno) a partir de columnas temporales.
    Por defecto, transforma: month, day, hour y minute.
    La columna 'year' se deja sin transformar.
    """
    def __init__(self, columns=None):
        """
        columns: diccionario que define para cada columna el valor máximo (p.ej. {"month": 12, "day": 31, "hour": 24, "minute": 60})
        Si se omite, se usan valores por defecto.
        """
        if columns is None:
            self.columns = {"month": 12, "day": 31, "hour": 24, "minute": 60}
        else:
            self.columns = columns

    def fit(self, X, y=None):
        # No es necesario aprender nada de X
        return self

    def transform(self, X):
        X_ = X.copy()
        for col, max_val in self.columns.items():
            if col in X_.columns:
                X_[f'{col}_sin'] = np.sin(2 * np.pi * X_[col].astype(float) / max_val)
                X_[f'{col}_cos'] = np.cos(2 * np.pi * X_[col].astype(float) / max_val)
        return X_

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OrdinalEncoder


status_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='NOT_IN_SERVICE')),
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore', categories=[['IN_SERVICE', 'MAINTENANCE', 'NOT_IN_SERVICE', 'PLANNED']])), # One-Hot Encoding
])

is_charging_station_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(sparse_output=False, drop='if_binary')),
])

ordinal_pipeline = Pipeline([
    ('encoder', OrdinalEncoder())
])

cyclic_pipeline = Pipeline([
    ('cyclic_transformer', CyclicalFeaturesTransformer())
])

In [4]:
preprocessor = ColumnTransformer(
    transformers=[
        ('status', status_pipeline, ['status']),
        ('is_charging_station', is_charging_station_pipeline, ['is_charging_station']),
        ('ordinal', ordinal_pipeline, ['month','day','hour']),
        ('cyclic', cyclic_pipeline, ['month','day','hour','minute']),
        ('year', 'passthrough', ['year']),
    ],
    remainder='passthrough'
)

In [5]:
df_merge = cargar_datos()
# transformed_data = preprocessor.fit_transform(df_merge)

/var/folders/yb/bj954x3n3xvfw__dvzkw3kb40000gn/T/ipykernel_16120/3027167522.py:45: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['last_reported'] = pd.to_datetime(df['last_reported'], unit='s', errors='coerce')


In [6]:
df_capacity = df_merge[['capacity', 'num_bikes_available', 'num_mechanical', 'num_ebike', 'num_docks_available']].copy()

df_capacity['diff_bike_types'] = df_capacity['num_mechanical'] + df_capacity['num_ebike'] - df_capacity['num_bikes_available']
df_capacity['diff_capacity_available'] = df_capacity['capacity'] - (df_capacity['num_bikes_available'] + df_capacity['num_docks_available'])

# Dime el total de diff_capacity_available / entre los diff_capacity_available que no son iguales a 0
df_capacity[df_capacity['diff_capacity_available'] == 0].shape[0] / df_capacity.shape[0]

0.27085833976558277

In [ ]:
df_merge.sort_values(by=['station_id','year', 'month', 'day', 'hour'], ascending=True, inplace=True)
df_merge.head()  


,station_id,year,month,day,hour,num_bikes_available,num_docks_available,num_mechanical,num_ebike,target,lat,lon,capacity
0,1,2024.0,4.0,30.0,21.0,20.000000,25.000000,19.0,1.0,0.543478,41.397978,2.180107,46
1,1,2024.0,4.0,30.0,22.0,25.250000,19.750000,20.0,6.0,0.429348,41.397978,2.180107,46
2,1,2024.0,4.0,30.0,23.0,32.333333,12.666667,24.5,9.0,0.275362,41.397978,2.180107,46
3,1,2024.0,5.0,1.0,0.0,25.916667,19.083333,16.5,6.5,0.414855,41.397978,2.180107,46
4,1,2024.0,5.0,1.0,1.0,18.416667,26.583333,15.0,4.0,0.577899,41.397978,2.180107,46


,station_id,year,month,day,hour,num_bikes_available,num_docks_available,num_mechanical,num_ebike,target,lat,lon,capacity
4,1,2024.0,5.0,1.0,1.0,18.416667,26.583333,15.0,4.0,0.577899,41.397978,2.180107,46
5,1,2024.0,5.0,1.0,2.0,16.750000,28.250000,14.0,2.0,0.614130,41.397978,2.180107,46
6,1,2024.0,5.0,1.0,3.0,17.000000,28.000000,16.0,1.0,0.608696,41.397978,2.180107,46
7,1,2024.0,5.0,1.0,4.0,17.166667,27.833333,17.0,0.0,0.605072,41.397978,2.180107,46
8,1,2024.0,5.0,1.0,5.0,18.666667,26.333333,19.0,0.0,0.572464,41.397978,2.180107,46


In [ ]:
#suposant que comencem per l'inici dun dia(elimino les 4 hores del dia anterior del mes anterior, ja que per defecte surten)
df_merge = df_merge.iloc[4:]
df_merge.head()  

#funcio per afegir les 4h anteriors
def crear_campos_lags(df):
    resultados = []
    # Agrupamos por station_id
    for station, grupo in df.groupby('station_id'):
        # Aseguramos el orden cronológico
        grupo = grupo.sort_values(by=['year', 'month', 'day', 'hour']).reset_index(drop=True)
        n = len(grupo)
        # Iteramos empezando en el índice 4 y avanzamos de 5 en 5
        for i in range(4, n, 5):
            if i - 4 >= 0:
                fila = grupo.loc[i].copy()
                # Agregamos los valores de 'target' de las 4 horas previas
                fila['4h_before'] = grupo.loc[i - 4, 'target']
                fila['3h_before'] = grupo.loc[i - 3, 'target']
                fila['2h_before'] = grupo.loc[i - 2, 'target']
                fila['1h_before'] = grupo.loc[i - 1, 'target']
                
                resultados.append(fila)
    return pd.DataFrame(resultados)

# Aplicamos la función al DataFrame df_merge
df_merge_final = crear_campos_lags(df_merge)

print(df_merge_final.head())  # Ver las primeras filas del nuevo DataFrame


    station_id    year  month  day  hour  num_bikes_available  \
4          1.0  2024.0    5.0  1.0   5.0            18.666667   
9          1.0  2024.0    5.0  1.0  10.0            18.750000   
14         1.0  2024.0    5.0  1.0  15.0             8.250000   
19         1.0  2024.0    5.0  1.0  20.0            12.166667   
24         1.0  2024.0    5.0  2.0   1.0            20.166667   

    num_docks_available  num_mechanical  num_ebike    target        lat  \
4             26.333333            19.0        0.0  0.572464  41.397978   
9             25.750000            17.5        1.0  0.559783  41.397978   
14            34.416667             5.0        3.0  0.748188  41.397978   
19            31.500000             9.0        3.0  0.684783  41.397978   
24            23.833333            11.0        9.0  0.518116  41.397978   

         lon  capacity  4h_before  3h_before  2h_before  1h_before  
4   2.180107      46.0   0.577899   0.614130   0.608696   0.605072  
9   2.180107      46